In [ ]:
%%bigquery
create or replace procedure `data-lake-290221.covid19_confirmed.create_locations`()
begin
create or replace table `data-lake-290221.covid19_confirmed.locations` as
select farm_fingerprint(concat(cast(latitude as string), cast(longitude as string))) as location_id, 
       province_state, country_region, latitude, longitude, location_geom
from covid19_confirmed.raw_cases;
end;

In [ ]:
%%bigquery
call `data-lake-290221.covid19_confirmed.create_locations`()

In [ ]:
%%bigquery
select * from data-lake-290221.covid19_confirmed.locations
limit 10

### Only run the beam pipeline locally through this notebook
### To pipeline run in Dataflow, use terminal window 

In [1]:
%run transpose_cases.py

/home/jupyter/transpose_cases.py:53: BeamDeprecationWarning: BigQuerySource is deprecated since 2.25.0. Use ReadFromBigQuery instead.
  bq_source = BigQuerySource(query=sql, use_standard_sql=True)


/home/jupyter/venv/lib/python3.7/site-packages/apache_beam/io/gcp/bigquery.py:1971: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  temp_location = pcoll.pipeline.options.view_as(
/home/jupyter/venv/lib/python3.7/site-packages/apache_beam/io/gcp/bigquery_file_loads.py:900: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  temp_location = p.options.view_as(GoogleCloudOptions).temp_location


In [1]:
%%bigquery
create or replace procedure `data-lake-290221.covid19_confirmed.create_aggregations`()
begin
  select location_id, country_region, province_state, date, avg_cases
   from
   (select l.location_id, l.province_state, l.country_region, c.date, avg(c.cases) as avg_cases
   from covid19_confirmed.daily_cases c join covid19_confirmed.locations l
   on c.location_id = l.location_id
   group by l.location_id, l.province_state, l.country_region, c.date)
   order by date;
end;

""


In [2]:
%%bigquery
call `data-lake-290221.covid19_confirmed.create_aggregations`()

,location_id,country_region,province_state,date,avg_cases
0,-8917363289061832566,China,Chongqing,2020-01-22,6.0
1,6390168571513050285,China,Hubei,2020-01-22,444.0
2,4378140026320891257,China,Beijing,2020-01-22,14.0
3,7068774372673742706,China,Tianjin,2020-01-22,4.0
4,2912167951723785902,China,Zhejiang,2020-01-22,10.0
...,...,...,...,...,...
87205,-4341312630839217445,China,Qinghai,2020-12-09,18.0
87206,-615446845178709331,Canada,Grand Princess,2020-12-09,183.5
87207,-615446845178709331,Canada,Diamond Princess,2020-12-09,183.5
87208,-615446845178709331,MS Zaandam,None,2020-12-09,183.5
